<a href="https://colab.research.google.com/github/grayzerguy/02-homework-solution/blob/master/Copy_of_zara_b2b2_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# התקנת ספריות
!pip install pandas openpyxl

import pandas as pd
import requests
import json
import time
from google.colab import files
# === הגדרות API ===
API_KEY = "4c78fa30-f723-4cdd-99ae-b4784750acbc"
SECRET_KEY = "69ad763b-025b-4598-8dea-924c1a0b98f7"
API_URL = "https://restapi.tapuzdelivery.com/api/delivery/create-new"

headers = {
    "Content-Type": "application/json",
    "Authorization": json.dumps({
        "api_key": API_KEY,
        "secret_key": SECRET_KEY
    })
}


# === שלב 1: העלאת קובץ המשלוחים ===
print("🔼 העלה את קובץ המשלוחים (כולל Shipping bulk ו-Store)")
uploaded = files.upload()
shipping_file = list(uploaded.keys())[0]
df_shipping = pd.read_excel(shipping_file, dtype=str)

# ניקוי שמות עמודות
df_shipping.columns = [col.strip() for col in df_shipping.columns]

if 'Shipping bulk' not in df_shipping.columns or 'Store' not in df_shipping.columns:
    raise ValueError("❌ קובץ לא מכיל עמודות נדרשות: 'Shipping bulk' ו-'Store'")

# שמירת שורות ייחודיות בלבד
df_shipping = df_shipping[['Shipping bulk', 'Store']].drop_duplicates()

# === שלב 2: טבלת סניפים קבועה ===
branch_data = [
    {"Store": "3035", "שם נמען": "זארה דיזינגוף סנטר", "יישוב": "תל אביב", "רחוב": "דיזינגוף", "מס": "50"},
    {"Store": "3013", "שם נמען": "זארה רמת אביב", "יישוב": "תל אביב", "רחוב": "אינשטיין", "מס": "40"},
    {"Store": "3209", "שם נמען": "זארה סי מול", "יישוב": "אשדוד", "רחוב": "הגדוד העיברי", "מס": "6"},
    {"Store": "3049", "שם נמען": "זארה חולון", "יישוב": "חולון", "רחוב": "גולדה מאיר", "מס": "7"},
    {"Store": "3070", "שם נמען": "זארה גרנד", "יישוב": "חיפה", "רחוב": "שמחה גולן", "מס": "53"},
    {"Store": "3012", "שם נמען": "זארה מלחה", "יישוב": "ירושלים", "רחוב": "אגודת ספורט בית\"ר", "מס": "1"},
    {"Store": "16364", "שם נמען": "זארה קיריון", "יישוב": "חיפה", "רחוב": "דרך עכו", "מס": "119"},
    {"Store": "3523", "שם נמען": "זארה גבעתיים", "יישוב": "גבעתייים", "רחוב": "יצחק רבין", "מס": "53"},
    {"Store": "3208", "שם נמען": "זארה רחובות", "יישוב": "רחובות", "רחוב": "בילו", "מס": "2"},
    {"Store": "3062", "שם נמען": "זארה הנגב", "יישוב": "באר שבע", "רחוב": "אלי כהן", "מס": "1"},
    {"Store": "3191", "שם נמען": "זארה מול ים", "יישוב": "אילת", "רחוב": "הטיילת", "מס": "7"},
    {"Store": "3389", "שם נמען": "זארה רננים", "יישוב": "רעננה", "רחוב": "המלאכה", "מס": "2"},
    {"Store": "3039", "שם נמען": "זארה איילון", "יישוב": "רמת גן", "רחוב": "אבא הלל", "מס": "300"},
    {"Store": "3637", "שם נמען": "זארה הזהב", "יישוב": "ראשון לציון", "רחוב": "דוד בחרוב", "מס": "21"},
    {"Store": "3086", "שם נמען": "זארה אבנת", "יישוב": "פתח תקווה", "רחוב": "זבוטיסקי", "מס": "7"},
    {"Store": "14538", "שם נמען": "זארה מודיעין", "יישוב": "מודעין", "רחוב": "לב העיר", "מס": "2"},
    {"Store": "4538", "שם נמען": "זארה מודיעין", "יישוב": "מודעין", "רחוב": "לב העיר", "מס": "2"},
    {"Store": "3947", "שם נמען": "זארה עזריאלי", "יישוב": "תל אביב", "רחוב": "מנחם בגין", "מס": "126"},
    {"Store": "9006", "שם נמען": "זארה ביג נצרת", "יישוב": "נצרת", "רחוב": "תאופיק זיאד", "מס": "53"},
    {"Store": "9032", "שם נמען": "זארה G כפר סבא", "יישוב": "כפר סבא", "רחוב": "ויצמן", "מס": "200"},
    {"Store": "1227", "שם נמען": "זארה הזהב", "יישוב": "ראשון לציון", "רחוב": "דוד סחרוב", "מס": "21"},
    {"Store": "367", "שם נמען": "זארה ממילא", "יישוב": "ירושלים", "רחוב": "ממילא", "מס": "1"},
    {"Store": "6415", "שם נמען": "זארה עיר ימים", "יישוב": "נתניה", "רחוב": "בני ברמן", "מס": "2"},
    {"Store": "6470", "שם נמען": "זארה גרנד", "יישוב": "באר שבע", "רחוב": "דוד טוביהו", "מס": "125"},
    {"Store": "11581", "שם נמען": "זארה גינדי", "יישוב": "תל אביב", "רחוב": "חשמונאים", "מס": "100"},
    {"Store": "12682", "שם נמען": "זארה מול החוף ויליג", "יישוב": "חדרה", "רחוב": "פרופ דן שכטמן", "מס": "2"},
    {"Store": "16515", "שם נמען": "זארה גלילות", "יישוב": "רמת השרון", "רחוב": "צומת גלילות", "מס": "1"},
    {"Store": "11835", "שם נמען": "זארה הום גינדי", "יישוב": "תל אביב", "רחוב": "חשמונאים", "מס": "100"},
    {"Store": "12371", "שם נמען": "זארה הום איילון", "יישוב": "רמת גן", "רחוב": "אבא הלל סילבר", "מס": "301"}
]
branches_df = pd.DataFrame(branch_data)

# === שלב 3: מיזוג הנתונים ===
merged_df = df_shipping.merge(branches_df, on='Store', how='left')
merged_df = merged_df.dropna(subset=['שם נמען'])

# === שלב 4: שליחה ל-API ===
for _, row in merged_df.iterrows():
    try:
        try:
            target_street_number = int(float(row['מס']))
        except:
            target_street_number = 0

        payload = {
            "sender_name": row['שם נמען'],
            "contact_customer_name": row['שם נמען'],
            "contact_customer_email": "test@zarazara.com",
            "delivery_type_code": 1,
            "amount": 1,
            "target_name": row['שם נמען'],
            "target_city": row['יישוב'],
            "target_street_name": row['רחוב'],
            "target_street_number": target_street_number,
            "floor": "",
            "entrance": "",
            "apartment": "",
            "target_phone": "0500000000",
            "barcode": row['Shipping bulk'],
            "govaina_amount": 0,
            "comments": "",
            "order_number": row['Store'],
            "target_email": "",
            "collection_address": {
                "target_city": "ראשון לציון",
                "target_street_name": "הרצל",
                "target_street_number": 20
            },
            "items": [
                {
                    "name": "product name",
                    "sku": "product sku",
                    "image_url": "",
                    "qty": 1,
                    "amount": 120
                }
            ]
        }

        response = requests.post(API_URL, headers=headers, json=payload, verify=False)
        if response.status_code == 200:
            print(f"✅ נשלח בהצלחה: {row['Shipping bulk']}")
        else:
            print(f"❌ שגיאה ({response.status_code}): {row['Shipping bulk']} - {response.text}")
    except Exception as e:
        print(f"🚫 שגיאת מערכת: {row['Shipping bulk']} - {e}")
    time.sleep(0.2)
